In [1]:
!pip install s3fs==2022.11.0 gputil==1.4.0

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 127.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 25.1 MB/s eta 0:00:00
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7410 sha256=8168aaf0471e7dc36ad0e22a325aa03297ca8897bf155236912e0783c326876f
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built gputil
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.3.0
    Uninstalling fsspec-2023.3.0:
      Successfully uninstalled fsspec-2023.3.0
ERROR: pip's dependency resolver does no

In [2]:
import os
import cudf
import GPUtil
import dask_cudf
import cupy as cp
import pandas as pd

In [9]:
#cudf_df = cudf.read_parquet('s3://bigdatateaching/nyctaxi-yellow-tripdata/2021/*.parquet')
df_list = []
for i in range(12):
    df = cudf.read_parquet(f"s3a://bigdatateaching/nyctaxi-yellow-tripdata/2021/yellow_tripdata_2021-{i+1:02d}.parquet")
    df_list.append(df)
cudf_df = cudf.concat(df_list)

In [10]:
cudf_df.shape

(30904308, 19)

In [11]:
cudf_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,<NA>
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,<NA>
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,<NA>
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,<NA>
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,<NA>


In [12]:
cudf_df['year'] = cudf.to_datetime(cudf_df['tpep_pickup_datetime']).dt.year
cudf_df['month'] = cudf.to_datetime(cudf_df['tpep_pickup_datetime']).dt.month
cudf_df['day'] = cudf.to_datetime(cudf_df['tpep_pickup_datetime']).dt.day
cudf_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,year,month,day
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,...,0.5,0.00,0.0,0.3,11.80,2.5,<NA>,2021,1,1
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,...,0.5,0.00,0.0,0.3,4.30,0.0,<NA>,2021,1,1
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,...,0.5,8.65,0.0,0.3,51.95,0.0,<NA>,2021,1,1
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,...,0.5,6.05,0.0,0.3,36.35,0.0,<NA>,2021,1,1
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,...,0.5,4.06,0.0,0.3,24.36,2.5,<NA>,2021,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214364,2,2021-12-31 23:18:02,2021-12-31 23:32:19,<NA>,3.99,<NA>,<NA>,145,107,0,...,0.5,5.80,0.0,0.3,26.52,<NA>,<NA>,2021,12,31
3214365,2,2021-12-31 23:46:00,2022-01-01 00:07:00,<NA>,5.96,<NA>,<NA>,236,158,0,...,0.5,2.85,0.0,0.3,29.20,<NA>,<NA>,2021,12,31
3214366,2,2021-12-31 23:46:00,2022-01-01 00:04:00,<NA>,4.99,<NA>,<NA>,48,13,0,...,0.5,5.23,0.0,0.3,29.50,<NA>,<NA>,2021,12,31
3214367,2,2021-12-31 23:53:00,2022-01-01 00:02:00,<NA>,1.36,<NA>,<NA>,75,75,0,...,0.5,2.00,0.0,0.3,16.00,<NA>,<NA>,2021,12,31


In [19]:
%time
cudf_df.groupby(['year', 'month', 'day']).agg({'trip_distance': "mean",
                                               'fare_amount': "mean",
                                               'mta_tax': 'mean',
                                               'tip_amount': 'mean',
                                               'total_amount': 'mean',
                                               'congestion_surcharge': 'mean'}).reset_index().sort_values(by=['year', 'month', 'day'],ascending=False)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


,year,month,day,trip_distance,fare_amount,mta_tax,tip_amount,total_amount,congestion_surcharge
349,2098,9,11,4.9500,22.500,0.5,0.000,25.800,2.5
229,2070,8,29,0.0000,0.000,0.0,0.000,0.000,0.0
39,2029,5,5,1.6900,7.500,0.5,2.160,12.960,2.5
316,2028,12,7,16.1500,52.000,0.5,0.000,63.100,2.5
303,2022,3,14,1.2600,6.000,0.5,2.055,11.355,2.5
...,...,...,...,...,...,...,...,...,...
279,2003,1,9,0.0000,2.500,0.5,0.010,4.310,0.0
154,2003,1,5,0.0000,0.000,0.0,0.000,0.000,0.0
354,2003,1,3,0.0000,0.000,0.0,0.000,0.000,0.0
258,2003,1,1,4.1325,14.375,0.5,1.000,19.175,2.5
